In [0]:
 !pip install kaggle

In [0]:
!pip install -U efficientnet

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

In [5]:

from zipfile import ZipFile
file_name = "../input/dogs-vs-cats-redux-kernels-edition/train.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [20]:
from zipfile import ZipFile
file_name = "../input/dogs-vs-cats-redux-kernels-edition/test.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [9]:

import cv2
import numpy as np
import pandas as pd
import os
from random import shuffle
from tqdm import tqdm

import tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,GlobalAveragePooling2D,Flatten,LSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report , confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import RMSprop


In [10]:
TRAIN_DIR = 'train'
TEST_DIR = 'test'
IMG_SIZE = 100
LR = 1e-3


MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR,'2conv-basic')


In [11]:
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat': return [1,0]
    elif word_label == 'dog': return [0,1]

In [12]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.resize(cv2.imread(path,cv2.IMREAD_COLOR),(IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy',training_data)
    return training_data

In [17]:
def create_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.resize(cv2.imread(path,cv2.IMREAD_COLOR),(IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
    np.save('test_data.npy',testing_data)
    return testing_data

In [14]:
train_data = create_train_data() 

100%|██████████| 25000/25000 [00:51<00:00, 487.38it/s]


In [21]:
test_data = create_test_data()


100%|██████████| 12500/12500 [00:25<00:00, 487.26it/s]


In [22]:
train_data[0] 

[array([[[221, 227, 238],
         [209, 211, 232],
         [211, 210, 244],
         ...,
         [244, 242, 248],
         [244, 242, 248],
         [245, 244, 246]],
 
        [[201, 200, 231],
         [203, 204, 232],
         [209, 209, 236],
         ...,
         [243, 243, 249],
         [243, 243, 249],
         [245, 244, 246]],
 
        [[200, 198, 234],
         [208, 206, 242],
         [210, 208, 243],
         ...,
         [238, 242, 247],
         [239, 243, 248],
         [244, 246, 247]],
 
        ...,
 
        [[196, 203, 220],
         [192, 200, 217],
         [188, 201, 217],
         ...,
         [113, 118, 143],
         [114, 119, 144],
         [114, 117, 138]],
 
        [[197, 205, 222],
         [195, 203, 220],
         [193, 201, 218],
         ...,
         [109, 116, 136],
         [112, 119, 139],
         [114, 120, 139]],
 
        [[195, 203, 220],
         [197, 205, 222],
         [197, 205, 222],
         ...,
         [111, 118, 138],
  

In [23]:
train = train_data[:-500]
test = train_data[-500:]

In [24]:
X_train = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
y_train = [i[1] for i in train]

X_test = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
y_test = [i[1] for i in test]

X_train = X_train / 255
X_test = X_test / 255



In [25]:
image_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.1,1.5],

)
# keras.preprocessing.image.ImageDataGenerator(
#     featurewise_center=False,
#      samplewise_center=False,
#       featurewise_std_normalization=False,
#        samplewise_std_normalization=False, 
#        zca_whitening=False,
#         zca_epsilon=1e-06, 
#         rotation_range=0, 
#         width_shift_range=0.0,
#          height_shift_range=0.0,
#           brightness_range=None,
#            shear_range=0.0,
#             zoom_range=0.0,
#              channel_shift_range=0.0,
#               fill_mode='nearest',
#                cval=0.0,
#                 horizontal_flip=False,
#                  vertical_flip=False,
#                   rescale=None,
#                    preprocessing_function=None,
#                     data_format='channels_last',
#                      validation_split=0.0,
#                       interpolation_order=1,
#                        dtype='float32')

In [26]:
type(X_train)


numpy.ndarray

In [27]:
train_generator = image_gen.flow(X_train,y_train,batch_size = 32)


val_generator = image_gen.flow(X_test,y_test,batch_size = 32)


In [0]:
# model = Sequential()

# model.add(LSTM(128, return_sequences = True, input_shape = (60, 5)))
# model.add(Dropout(0.2))

# model.add(LSTM(128, return_sequences = True))
# model.add(Dropout(0.2))

# model.add(LSTM(80, return_sequences = True))
# model.add(Dropout(0.2))

# model.add(LSTM(40))
# model.add(Dropout(0.2))


# model.add(Dense(units = 1))

In [0]:
# model.compile(optimizer='adam', loss = 'mean_squared_error',metrics = ['accuracy'])

In [28]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
def model(input_shape, classes):
    pretrained_model = tensorflow.keras.applications.vgg16.VGG16(input_shape=input_shape, weights='imagenet', include_top=False)
    pretrained_model.trainable = False
    model = Sequential()
    for layer in pretrained_model.layers:
        model.add(layer)
    model.add(GlobalAveragePooling2D())
    model.add(Flatten())
    model.add(Dense(128, activation='softmax'))
    model.add(Dense(64, activation='softmax'))
    model.add(Dense(classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])
    return model
# each pic has been resized to 100x100, and with 3 channels (RGB)
input_shape = (100,100,3)
classes = 2
model = model(input_shape, classes)
model.summary()

58892288/58889256 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0         
_________________________________________________________

In [30]:
model.fit_generator(train_generator,
                    epochs=5,
                    verbose=2,
                    validation_data = val_generator,
                    use_multiprocessing=True,
                    workers=2)


Train for 766 steps, validate for 16 steps
Epoch 1/5
766/766 - 71s - loss: 0.3334 - accuracy: 0.8920 - val_loss: 0.3290 - val_accuracy: 0.8920
Epoch 2/5
766/766 - 71s - loss: 0.3192 - accuracy: 0.8960 - val_loss: 0.3073 - val_accuracy: 0.9020
Epoch 3/5
766/766 - 71s - loss: 0.3163 - accuracy: 0.8976 - val_loss: 0.3084 - val_accuracy: 0.9100
Epoch 4/5
766/766 - 71s - loss: 0.3082 - accuracy: 0.9018 - val_loss: 0.3112 - val_accuracy: 0.9060
Epoch 5/5
766/766 - 72s - loss: 0.3127 - accuracy: 0.9003 - val_loss: 0.3189 - val_accuracy: 0.8980


In [ ]:
cd ../input/dogs-vs-cats-redux-kernels-edition

In [33]:
with open('../input/dogs-vs-cats-redux-kernels-edition/submission-file.csv','w') as f:
    f.write('id,label\n')

FileNotFoundError: [Errno 2] No such file or directory: '../input/dogs-vs-cats-redux-kernels-edition/output/submission-file.csv'

In [ ]:
kaggle competitions submit -c dogs-vs-cats-redux-kernels-edition -f submission.csv -m "Message"

In [0]:
with open('submission-file.csv','w') as f:
    for data in tqdm(test_data):
        img_num = data[1]
        img_data = data[0]
        orig = img_data
        data = img_data.reshape(-1,IMG_SIZE,IMG_SIZE,3)
        model_out = model.predict([data])[0]
        f.write('{},{}\n'.format(img_num,model_out[1]))